# 心理咨询论坛问答语料(无标签平面文本) - 制作迁徙学习数据集

为 NVIDIA/Megatron-LM 准备该领域的 finetune 数据集

## 说明

- 用问题与其多个答案中的一个反复组合，形成多个问题-答案对
- 使用 tag 或者问题中的关键字，选取一个类型的问答数据作为语料

来源语料格式举例：

```json
{'topics': ['婚姻'],
 'title': [],
 'text': ['其实无论我们遇见谁和谁相恋，无论成功与否都是过程和经历，即便是失败也不要沮丧，我们要回头在那失败的地方我们学到了什么，并且有什么美好的回忆留在心里，在人生道路上，只要我们认真对待感情，正正经经的恋爱，就算被辜负或者遇见错的人而失望，但我相信认真对待生活的人一定会有收获的，再困难再痛苦事情都会过去的，你依然是你，天也不会塌，经历会让我们成长，更懂得生活的意义，才会有精彩的人生。',
  '会有那么一个人会看见你的内在，拥抱你真挚的心。',
  '我说的对么?'],
 'answers': [['是的，', '做好自己，', '勇敢面对一切快乐和痛苦，', '一切源于我们的内心，', '修炼自己']]}
```

形成的平面带有分隔符的QA语料是：

```js
    plain_text(title) + "<sep>"
  + plain_text(text) + plain_text(slave_text) + "<sep>"
  + "<sep>"
  + "<|endoftext|>" + plain_text(answer)
```

将输出的语料存放在 JSON Lines 文件中，文本内容放在 `text` 属性中


## CD

切换到工作目录(**按实际情况，勿照搬下面的 Cell**)

In [1]:
%cd ..

/home/Public/Megatron-LM


## Importings

In [2]:
import os
import json
import random
from copy import copy
from contextlib import closing, ExitStack
import fileinput
from functools import partial
from glob import glob, iglob
from itertools import chain, repeat
from multiprocessing import Pool

import pandas as pd
import sentencepiece as spm
from tqdm.auto import tqdm, trange
from slugify import slugify

from IPython.display import display

## Tokenizer

使用与 Pretrained-Model 同样的 tokenizer

In [3]:
%%time

from data_utils.tokenization import SentencePieceTokenizer, make_tokenizer


TOKENIZER_MODEL_PATH='./data/spm/gpt2_huamei_corpus_bpe_32k_v2.model'

sp = spm.SentencePieceProcessor()
sp.load(TOKENIZER_MODEL_PATH)

print('SentencePiece 内置的特殊标记:')
for i in range(8):
  print(i, sp.id_to_piece(i), sp.is_control(i))
del sp
print()

print('Wrapped tokenizer ...')
tokenizer = make_tokenizer(
    SentencePieceTokenizer,
    None,
    model_path=TOKENIZER_MODEL_PATH
)

# tokenizer.command_tokens
for tok in tokenizer.command_name_map.values():
    print(f'{tok.Id}: {tok.token}')

# 需要的特殊标记：
SEP = '<sep>'
START_OF_ANSWER = '<|endoftext|>'

for piece in (SEP, START_OF_ANSWER):
    assert piece == tokenizer.IdToToken(tokenizer.TokenToId(piece))
    assert piece == tokenizer.IdToToken(tokenizer.EncodeAsIds(piece).tokenization[-1])


SentencePiece 内置的特殊标记:
0 <pad> True
1 <unk> False
2 <bos> True
3 <eos> True
4 <sep> False
5 <cls> False
6 <|endoftext|> False
7 一个 False

Wrapped tokenizer ...
0: <pad>
1: <eos>
2: <bos>
3: <unk>
4: <sep>
5: <L2R>
6: <ENC>
7: <MASK>
CPU times: user 733 ms, sys: 139 ms, total: 872 ms
Wall time: 891 ms


## Corpus

In [4]:
corpus_files = glob('/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/*.json')

In [5]:
print('语料文件:')
display(corpus_files)
print()

with fileinput.input(corpus_files) as reader:
    total = sum(1 for _ in tqdm(reader, unit='line'))

print(f'总行数: {total:,d}')

语料文件:


['/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/valid_xinli_qax.json',
 '/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/train_xinli_qax.json',
 '/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/test_xinli_qax.json']


总行数: 187,156


现在预览语料数据格式，随机选择一个：

In [6]:
n = random.randint(0, total)

with fileinput.input(corpus_files) as reader:
    for i, line in tqdm(zip(range(n), reader), total=n):
        if i+1 < n:
            continue
        data = json.loads(line)
        display(i)
        display(data)


20803

{'topics': ['性心理', '性心理'],
 'title': ['哇啊啊啊啊啊，我是不是变态啊！'],
 'text': ['咳咳咳，今天我朋友给我看A片我看了一点就觉得无趣，还有点恶心。',
  '但是我看耽美的时候我特别感觉兴奋，还有点小激动。',
  '我是女的哦！',
  '（当然我讨厌百合，也觉得恶心），我前世是不是一个男的，而且还是骨灰级断袖啊。',
  '求卍解'],
 'answers': [['你好，',
   '我是小媒。',
   '腐女，',
   '指支持BL，',
   '赞美男男爱情的女性。',
   '这种心理可能产生于这几个方面。',
   '1防御心理。',
   '因为家庭教育中对于性的教育比较缺乏，',
   '并且存在一种“性是禁区”的思想。',
   '当我们为了避免男女之间的性，',
   '渐渐将目光转向男性之间。',
   '2嫉妒心理。',
   '既然得不到男神，',
   '那就让男神和男神在一起吧！',
   '用叔本华来解释腐女们的世界：基情虽是她们看到的表象，',
   '背后恐怕是难以填埋的欲望。',
   '她们的原则就是，',
   '宁可美男出双入对，',
   '不可让其落入敌手。',
   '既然男人有后宫情结，',
   '常幻想坐拥佳丽三千，',
   '女人自然也有幻想左拥右抱帅哥的邪念。',
   '3观察者心理。',
   '由于成长经历或是文化熏陶，',
   '男女之间的性被看为是肮脏的。',
   '而在欣赏耽美作品时，',
   '可以以一种更加纯粹的观察者角度来看爱情、性。',
   '我身边也有一些腐女，',
   '大多都是从青春期开始，',
   '到大学还腐的已经是少数。',
   '即使是，',
   '也能清楚地分清想象和现实，',
   '只将耽美放在想象里，',
   '不会影响现实生活。',
   '所以楼主不用太担心~过了这个时期就会好起来。',
   '如果的确很担心的话，',
   '也可以咨询专业咨询师哦~（以上部分内容来自百度）']]}

## 采样

### 采样条件

我们应：

- 选取问题+回答构成输入样本
- 一个问题可以对应多个回答
- 不要太短的
- 不要太长的
- 指定的主题

在此基础上，随机乱序选用若干样本用于今次的开发

- 主题

In [7]:
TOPIC = '婚姻'

- 长度限制定义

In [8]:
MIN_QUESTION_LENGTH = 16
MAX_QUESTION_LENGTH = 1024
MIN_ANSWER_LENGTH = 32
MAX_LENGTH = 2048

- 样本过滤函数

In [9]:
def extract_samples(line):
    line = line.strip()
    if not line:
        return []
    data = json.loads(line)
    result = []
#     # 主题不对的不要！
    topics = data.get('topics', []) 
    if TOPIC not in topics:
        return []
    # 没有回答的不要
    if not data.get('answers'):
        return []
    # 问题:
    title = ''.join(data['title']).strip()
    text = ''.join(data['text']).strip()
    # 既没有问题标题又没有问题内容的不要
    if not title and not text:
        return []
    n_title = len(tokenizer.EncodeAsIds(title))
    n_text = len(tokenizer.EncodeAsIds(text))
    # 太短不要
    if n_title + n_text < MIN_QUESTION_LENGTH:
        return []
    # 太长不要
    if n_title + n_text > MAX_QUESTION_LENGTH:
        return []
    # 回答：
    question_string = title + SEP + text + SEP
    # 答案
    for answer in data.get('answers', []):
        text = ''.join(answer).strip()
        length = len(tokenizer.EncodeAsIds(text))
        # 太短不要
        if length < MIN_ANSWER_LENGTH:
            continue
        # 太长不要
        if n_title + n_text + length > MAX_LENGTH:
            continue
        result.append(question_string + SEP + START_OF_ANSWER + text)
    #
    return result

### 进行采样

In [10]:
samples = []

with Pool() as pool, fileinput.input(corpus_files) as reader:
    mapper = pool.imap_unordered(
        extract_samples,
        tqdm(reader, total=total),
        chunksize=min(1024, total//os.cpu_count()+1)
    )
    for data_list in tqdm(mapper, total=total):
        samples.extend(data_list)

print(f'样本数量: {len(samples):,d}')



样本数量: 49,810


### 预览一条采样结果数据

In [17]:
s = random.choice(samples)
print(s)

老婆出轨了，我该怎么办?<sep>我很想和她分手，可是我们有两个孩子，分手了孩子怎么办？不分手又难受，恨这对狗男女的冲动，我该怎么办？很难受<sep><sep><|endoftext|>这真是一个糟糕的经历，哥们儿，先等等自己，允许自己痛苦一阵子，再说怎么办。内心受伤了，先疗愈自己的伤口，如果带着伤，还去奋战，那受伤的伤口，流血更多，内心更痛。


## 保存结果

### 输出目录

In [22]:
OUTPUT_DIR = os.path.abspath(
    os.path.join(
        'data',
        'xinliqa-{0}'.format(slugify(TOPIC, separator='_')),
    )
)

print(f'输出目录: {OUTPUT_DIR}')
os.makedirs(OUTPUT_DIR, exist_ok=True)

输出目录: /home/Public/Megatron-LM/data/xinliqa-hun_yin


### 不划分数据集保存

乱序后直接保存所有输出样本到文件

In [12]:
dst_file = os.path.join(
    OUTPUT_DIR,
    'qa.all.json'
)

print(f'保存所有符合条件的样本到文件: {dst_file}')

with open(dst_file, 'w') as fp:
    random.shuffle(samples)
    for txt in tqdm(samples):
        print(json.dumps({'text': txt}, ensure_ascii=False), file=fp)


保存所有符合条件的样本到文件: /home/Public/Megatron-LM/data/xinliqa-hun_yin/qa.all.json


检查文件:

In [13]:
!du -lh {dst_file}
!wc -l {dst_file}

56M	/home/Public/Megatron-LM/data/xinliqa-hun_yin/qa.all.json
49810 /home/Public/Megatron-LM/data/xinliqa-hun_yin/qa.all.json


### 划分数据集保存

更复杂的情况，划分数据集再保存

根据实际的样本数量，划分为 train/val/test，其样本数量应根据采样情况进行规定

In [23]:
parts = {
    'train': 45_000,
    'val': 2_000,
    'test': 2_000,
}

print('数据集划分：')
display(parts)

assert sum(parts.values()) <= len(samples)

数据集划分：


{'train': 45000, 'val': 2000, 'test': 2000}

#### 随机选择+洗牌

随机种子:

In [24]:
%%time

seeds = []
seeds.extend(chain.from_iterable(
    repeat(k, v) for k, v in parts.items()
))
seeds.extend(repeat(None, len(samples)-sum(parts.values())))
random.shuffle(seeds)

assert len(seeds) == len(samples)

CPU times: user 51.5 ms, sys: 81 µs, total: 51.6 ms
Wall time: 50.6 ms


#### 写目标文件

In [28]:
dst_files_dict = {
    k: os.path.join(OUTPUT_DIR, f'qa.{k}.json')
    for k in parts.keys()
}

print('输出文件:')
display(dst_files_dict)

with ExitStack() as stack, fileinput.input(corpus_files) as reader:
    files = {
        k: stack.enter_context(open(v, 'w'))
        for k, v in dst_files_dict.items()
    }
    for seed, string in tqdm(zip(seeds, samples), total=len(samples)):
        fp = files.get(seed)
        if not fp:
            continue
        print(
            json.dumps({'text': string}, ensure_ascii=False),
            file=fp
        )

输出文件:


{'train': '/home/Public/Megatron-LM/data/xinliqa-hun_yin/qa.train.json',
 'val': '/home/Public/Megatron-LM/data/xinliqa-hun_yin/qa.val.json',
 'test': '/home/Public/Megatron-LM/data/xinliqa-hun_yin/qa.test.json'}

检查输出文件

In [34]:
s = ' '.join(dst_files_dict.values())

!wc -l {s}
print()

!du -hc {s}
print()

   45000 /home/Public/Megatron-LM/data/xinliqa-hun_yin/qa.train.json
    2000 /home/Public/Megatron-LM/data/xinliqa-hun_yin/qa.val.json
    2000 /home/Public/Megatron-LM/data/xinliqa-hun_yin/qa.test.json
   49000 总用量

51M	/home/Public/Megatron-LM/data/xinliqa-hun_yin/qa.train.json
2.2M	/home/Public/Megatron-LM/data/xinliqa-hun_yin/qa.val.json
2.3M	/home/Public/Megatron-LM/data/xinliqa-hun_yin/qa.test.json
55M	总用量



## 测试数据加载

In [4]:
with open('/home/Public/Megatron-LM/data/xinli_qa_artical-hunyin/qa.test.json') as fp:
    for line in fp:
        data = json.loads(line)
        text = data['text']

text

'准备要结婚了，可是发现自己原来很恐婚，没有安全感<sep>今年28，和男友相恋一年，打算明年结婚，可是随着婚期临近遇到很多事情，所以对于婚姻越来越恐惧。其实在很早单身的时候就对婚姻有一种模糊的恐惧感，但是也没有太在意，就是怕结婚之后对方会突然死掉，和好朋友说了之后她们觉得我想太多了，我自己也觉得这个想法不切实际。后来有了男友，父母并不满意，但是也没有要求必须分开，她们保留意见。现在打算要结婚，涉及到一些金钱方面的问题，具体就是男友不出彩礼不买钻戒，单独给我和我家的东西他都很排斥，他说因为没钱，但是买了房付了首付，装修好了，以后贷款他还，父母对此很不满并且预言了我以后生活的种种不幸福，我尝试和男友还有父母沟通过，但是失败了。现在父母的不满意和预言的不幸福，男友的坚硬态度，我父母对男友的不满，种种压力让我的恐惧感又出现了，我该怎么办<sep><sep><|endoftext|>你在恐惧什么，你没说清楚。是对未来生活的未知恐惧还是对男友的认知和行为不匹配时的恐惧还是对家人和对方家人矛盾的恐惧还是对自己能否幸福怀疑的恐惧具体点 铁汁'

In [7]:
for id_ in tokenizer.EncodeAsIds(text):
    print(id_, tokenizer.DecodeIds([id_]))

8787 
363 准备
8826 要
725 结婚
8792 了
8785 ,
194 可是
6662 发现自己
638 原来
8879 很
9790 恐
9461 婚
8785 ,
20 没有
4743 安全感
12 <sep>
680 今年
1244 28
8785 ,
8828 和
2550 男友
8966 相
9801 恋
646 一年
8785 ,
1479 打算
4279 明年
725 结婚
8785 ,
194 可是
811 随着
9461 婚
9064 期
9731 临
9195 近
887 遇到
107 很多
356 事情
8785 ,
64 所以
204 对于
1255 婚姻
643 越来越
1884 恐惧
8788 。
156 其实
8796 在
8879 很
9253 早
4627 单身
76 的时候
8814 就
8847 对
1255 婚姻
1737 有一种
3820 模糊
8786 的
1884 恐惧
8959 感
8785 ,
61 但是
590 也没有
8998 太
3850 在意
8785 ,
29 就是
9391 怕
725 结婚
146 之后
462 对方
8830 会
327 突然
9115 死
9622 掉
8785 ,
8828 和
6903 好朋友
783 说了
146 之后
636 她们
75 觉得
364 我想
4720 太多了
8785 ,
2224 我自己
8824 也
75 觉得
31 这个
899 想法
8791 不
9318 切
472 实际
8788 。
246 后来
611 有了
2550 男友
8785 ,
365 父母
361 并不
2208 满意
8785 ,
61 但是
590 也没有
291 要求
286 必须
4331 分开
8785 ,
636 她们
3191 保留
1087 意见
8788 。
42 现在
1479 打算
8826 要
725 结婚
8785 ,
2411 涉及
8818 到
166 一些
4154 金钱
190 方面
467 的问题
8785 ,
856 具体
29 就是
2550 男友
1096 不出
9903 彩
9627 礼
8791 不
9277 买
10681 钻
10496 戒
8785 ,
3804 单独
474 给我
1314 和我
8861 家
5